In [165]:
# import the libraries
import time
import re
import random as rd
import pandas as pd
import numpy as np
import collections
from tqdm import tqdm
import pickle
import datetime
from dateutil.relativedelta import relativedelta

import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

from newspaper import Article

In [166]:
# Driver setting
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
headers = {
    "User-Agent": user_agent
}

def setWebdriver():
    service = ChromeService(executable_path=ChromeDriverManager().install())    # 크롬 드라이버 최신 버전 설정

    options = ChromeOptions()
    options.add_argument('user-agent=' + user_agent)
    options.add_argument('--start-maximized') #브라우저가 최대화된 상태로 실행됩니다.
    # options.add_argument('headless') #headless모드 브라우저가 뜨지 않고 실행됩니다.
    #options.add_argument('--window-size= x, y') #실행되는 브라우저 크기를 지정할 수 있습니다.
    #options.add_argument('--start-fullscreen') #브라우저가 풀스크린 모드(F11)로 실행됩니다.
    #options.add_argument('--blink-settings=imagesEnabled=false') #브라우저에서 이미지 로딩을 하지 않습니다.
    options.add_argument('--mute-audio') #브라우저에 음소거 옵션을 적용합니다.
    options.add_argument('incognito') #시크릿 모드의 브라우저가 실행됩니다.
    driver = webdriver.Chrome(service=service, options=options)

    return driver

In [171]:
# 원하는 횟수만큼 Scroll down

def scrollDown(driver, num: int): 
    
    for _ in tqdm(range(0, num), desc='Scrolling Down...', leave='False'):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(rd.uniform(0.1, 0.3))    

    return

In [169]:
# start_date부터 end_date까지 3개월 단위로 나누기

def getPeriods(start_date, end_date):
    
    periods = []
    while (start_date < end_date):
        if (start_date + relativedelta(months=3) > end_date):
            periods.append([start_date.strftime("%Y.%m.%d"), end_date.strftime("%Y.%m.%d")])

        else:
            periods.append([start_date.strftime("%Y.%m.%d"), (start_date + relativedelta(months=3)).strftime("%Y.%m.%d")])
        start_date += relativedelta(months=3)
    # print(periods)

    return periods

In [176]:
# 언론사 세팅

def setCompany():

    company_names = ["YTN", "JTBC", "MBC", "SBS", "국민일보", "매일경제", "조선일보", "중앙일보", "아시아경제", "한국경제", "KBS", "한겨례", "경향신문"]
    company_codes = {
        "YTN": "1052",
        "JTBC": "1437",
        "MBC": "1214",
        "SBS": "1055",
        "국민일보": "1005",
        "매일경제": "1009",
        "조선일보": "1023",
        "중앙일보": "1025",
        "아시아경제": "1277",
        "한국경제": "1015",
        "KBS": "1056",
        "한겨례": "1028",
        "경향신문": "1032",
    }

    return company_names, company_codes

In [173]:
# 기간 세팅 (3개월 단위로 나누기)

start_date = datetime.datetime.strptime("2020.09.01", "%Y.%m.%d")
end_date = datetime.datetime.strptime("2023.12.31", "%Y.%m.%d")
periods = getPeriods(start_date, end_date)

periods[0][0] = "2020.09.02"
print(periods)

[['2020.09.02', '2020.12.01'], ['2020.12.01', '2021.03.01'], ['2021.03.01', '2021.06.01'], ['2021.06.01', '2021.09.01'], ['2021.09.01', '2021.12.01'], ['2021.12.01', '2022.03.01'], ['2022.03.01', '2022.06.01'], ['2022.06.01', '2022.09.01'], ['2022.09.01', '2022.12.01'], ['2022.12.01', '2023.03.01'], ['2023.03.01', '2023.06.01'], ['2023.06.01', '2023.09.01'], ['2023.09.01', '2023.12.01'], ['2023.12.01', '2023.12.31']]


In [197]:
driver = setWebdriver()

In [136]:
company_code = companies[companies_name[0]]
query = querys[0]
start_date = periods[0][0]    # 시작 날짜
end_date = periods[0][1]    # 끝 날짜

In [137]:
url = f'https://search.naver.com/search.naver?where=news&query={query}&sm=tab_clk.jou&sort=0&photo=0&field=0&pd=3&ds={start_date}&de={end_date}&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked={company_code}&nso=so%3Ar%2Cp%3Afrom20210520to20240520&is_sug_officeid=0&office_category=0&service_area=0'
print(url)

https://search.naver.com/search.naver?where=news&query=국민의힘&sm=tab_clk.jou&sort=0&photo=0&field=0&pd=3&ds=2020.09.02&de=2020.12.01&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1052&nso=so%3Ar%2Cp%3Afrom20210520to20240520&is_sug_officeid=0&office_category=0&service_area=0


In [198]:
driver.get(url)

In [199]:
# 아래로 스크롤 할 수 있을 만큼 스크롤 다운
scrollDown(driver, 99)

Scrolling Down...: 100%|██████████| 99/99 [00:21<00:00,  4.66it/s]


In [200]:
news_list = driver.find_elements(By.CLASS_NAME, "news_tit")

In [201]:
print(len(news_list))

1000


In [203]:
news_url = news_list[0].get_attribute('href')
print(news_url)

https://www.ytn.co.kr/_ln/0101_202011301050011782


In [204]:
article = Article(news_url, language="ko")

In [ ]:
current_url = news_url
article = Article(current_url, language="ko")
article.download()
article.parse()

date = article.publish_date;    print(f'date: {date}')
authors = article.authors;      print(f'authors: {authors}')
title = article.title;  print(f'title: {title}')
text = article.text;    print(f'text: {text}')
print(f'{"-"*200}')

# dates.append(date)
# titles.append(title)
# texts.append(text)

In [191]:
def main():
    
    # 언론사 세팅
    company_names, comapny_codes = setCompany()

    # 기간 세팅 (3개월 단위로 나누기)
    start_date = datetime.datetime.strptime("2020.09.01", "%Y.%m.%d")
    end_date = datetime.datetime.strptime("2023.12.31", "%Y.%m.%d")
    periods = getPeriods(start_date, end_date)

    # 검색어 세팅
    querys = ["국민의힘", "더불어민주당"]
    
    
    driver = setWebdriver()

    for comany in tqdm(company_names, desc="Companies..."):
        company_code = comapny_codes[comany]

        for period in periods:
            start_date = period[0]
            end_date = period[1]

            for query in querys:
                url = f'https://search.naver.com/search.naver?where=news&query={query}&sm=tab_clk.jou&sort=0&photo=0&field=0&pd=3&ds={start_date}&de={end_date}&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked={company_code}&nso=so%3Ar%2Cp%3Afrom20210520to20240520&is_sug_officeid=0&office_category=0&service_area=0'

                driver.get(url)
                time.sleep(1)

                scrollDown(driver, 99)  # 아래로 스크롤 할 수 있을 만큼 스크롤 다운

                news_list = driver.find_elements(By.CLASS_NAME, "news_tit")

                for news in news_list:
                    news_url = news.get_attribute('herf')

                    driver.execute_script(f"window.open('{news_url}');")     # 새 탭 열기
                    driver.switch_to.window(driver.window_handles[1])       # 새 탭으로 이동

                    article = Article(current_url, language="ko")
                    article.download()
                    article.parse()

                    date = article.publish_date;    print(f'date: {date}')
                    authors = article.authors;      print(f'authors: {authors}')
                    title = article.title;  print(f'title: {title}')
                    text = article.text;    print(f'text: {text}')
                    print(f'{"-"*200}')
                    
                    time.sleep(1)
                    driver.close()                                          # 현재 탭 닫기
                    driver.switch_to.window(driver.window_handles[0])       # 0번째 탭으로 이동

In [ ]:
if __name__ == "__main__":
    main()

In [195]:
# 첫 번째 탭으로 이동
driver.switch_to.window(driver.window_handles[0])

news_list = driver.find_elements(By.CLASS_NAME, 'news_tit')
print(len(news_list))

0


In [206]:
df = pd.DataFrame(columns=["Date", "Title", "Text"])
df

,Date,Title,Text


In [30]:
df = pd.read_csv('../data/MBC.csv')
df

,Unnamed: 0,Query,Date,Title,Text
0,0,국민의힘,2020-12-01 17:40:51+09:00,"경찰청장, 국민의힘 초선-경찰 대치에 ""매끄럽지 못했다""",전체재생\n\n김창룡 경찰청장은 청와대 연풍문 앞에서 벌어진 국민의힘 초선의원들과 ...
1,1,국민의힘,2020-11-30 10:05:04+09:00,"국민의힘 ""최재성, 코로나 방역수칙상 못만난다더니 조기축구""",전체재생\n\n지난 27일부터 청와대 앞에서 릴레이 1인 시위를 벌이고 있는 국민의...
2,2,국민의힘,2020-11-30 13:46:31+09:00,"국민의힘, 재보궐선거 공약개발단 구성…추경호·지상욱 등 참여",전체재생\n\n국민의힘이 내년 서울시장과 부산시장 보궐선거의 공약을 개발할 '4·7...
3,3,국민의힘,2020-11-30 17:48:27+09:00,"국민의힘, 윤호중 법사위원장 징계안 제출…""인격모독""",전체재생\n\n국민의힘 의원들이 오늘 국회 의안과에 더불어민주당 소속 윤호중 법제사...
4,4,국민의힘,2020-12-01 10:27:13+09:00,"국민의힘 초선 의원들 ""1인시위 할수록 청와대 불통 느껴""","전체재생\n\n국민의힘 초선 의원들이 ""청와대 앞에서 1인 시위를 하면 할수록 청와..."
...,...,...,...,...,...
27530,27530,더불어민주당,NaN,"[시선집중] 이상민 ""당 뜯어고치지 못할 바엔 깔끔한 결별.. 국민의힘? 모든 가능...",\n\n\n\n■ 방송 : MBC 라디오 표준FM 95.9MHz <김종배의 시선집중...
27531,27531,더불어민주당,NaN,"[시선집중] 홍익표 “이낙연 신당, 가능성 높지 않아. 내년 총선 당에서 역할하셔야”",\n\n\n\n■ 방송 : MBC 라디오 표준FM 95.9MHz <김종배의 시선집중...
27532,27532,더불어민주당,NaN,"[시선집중] 홍익표 “이동관, 사퇴 ‘당한’ 것. 용산 주변에서 오래전부터 얘기 나와”",\n\n\n\n■ 방송 : MBC 라디오 표준FM 95.9MHz <김종배의 시선집중...
27533,27533,더불어민주당,NaN,"[시선집중] 장동혁 “與, 준예산 정국 만들어 야당 탓? 합의 원만한데.. 이재명 ...",\n\n\n\n■ 방송 : MBC 라디오 표준FM 95.9MHz <김종배의 시선집중...


In [28]:
df.isnull().sum()

Unnamed: 0     0
Query          0
Date          14
Title         14
Text          20
dtype: int64

In [5]:
import pandas as pd
tmp1 = pd.DataFrame(columns=["Query", "Date", "Title", "Text"])

In [18]:

tmp3 = pd.DataFrame({"Query": "123", "Date": "12", "Title": '123', "Text": '123'}, index=([len(tmp)]))
tmp = pd.concat([tmp, tmp3])

In [19]:
tmp

,Query,Date,Title,Text
0,123,12,123,123
0,123,12,123,123
2,123,12,123,123
3,123,12,123,123
